In [5]:
val input = readln()
val name = if (input.isNotBlank()) input else "Kotlin"
println("Hello, $name!")

org.jetbrains.kotlinx.jupyter.exceptions.ReplInterruptedException: The execution was interrupted

In [ ]:
// $を入れたければエスケープシーケンスを入れる
println("\$100の支払いが発生しました")

In [ ]:
val name = readln()
if (name.isNotBlank()) {
    println("Hello, ${name.length}-letter person!")
}

In [ ]:
// Stringテンプレートとif式の条件分岐を合わせれば、より簡潔に書ける
val name = readln()
println("Hello ${if (name.isBlank()) "someone" else name}!")

In [ ]:
// p35, listing 2.4
class Person(
    val name: String,
    var isStudent: Boolean
)

val inuverse = Person("Inuverse", true)
println("Person name: ${inuverse.name}")
println("Is the person student?: ${inuverse.isStudent}")

println("===== ${inuverse.name} graduates as a Ph.D. from XXX university. =====")
inuverse.isStudent = false

println("Person name: ${inuverse.name}")
println("Is the person student?: ${inuverse.isStudent}")

// inuverse.name = "nukoverse" // Val cannot be reassigned

In [ ]:
// p37
class Rectangle(val height: Int, val width: Int) {
    val isSquare: Boolean
        get() { // property getter declaration
            return height == width
        }
}

val rectangle = Rectangle(3, 3)
println("height: ${rectangle.height}")
println("height: ${rectangle.width}")
println("height: ${rectangle.isSquare}")

In [ ]:
class Rectangle(val height: Int, val width: Int) {
    val isSquare: Boolean
        get() = height == width // onelinerで書けるものは中括弧`{}`する必要ない
}
val rectangle = Rectangle(3, 3)
println("height: ${rectangle.height}")
println("height: ${rectangle.width}")
println("height: ${rectangle.isSquare}")

In [ ]:
// p37 Rectangleのダメな例
class XRectangle(
    val height: Int,
    val width: Int,
    var isSquare: Boolean
)

val xrectangle = XRectangle(3, 3, true)
println("height: ${xrectangle.height}")
println("height: ${xrectangle.width}")
println("height: ${xrectangle.isSquare}")


In [14]:
class Matrix(
    val rows: Int,
    val cols: Int,
    val data: DoubleArray
) {
    val isSquare: Boolean
        get() = rows == cols

    operator fun get(i: Int, j: Int): Double {
        return data[i * cols + j]
    }

    fun transpose(): Matrix {
        val newData = DoubleArray(data.size) { i ->
            data[(i % rows) * cols + (i / rows)]
        }
        return Matrix(cols, rows, newData)
    }
}

val matrixData = doubleArrayOf(
    1.0, 2.0, 3.0,
    4.0, 5.0, 6.0,
    7.0, 8.0, 9.0
)
val matrix = Matrix(3, 3, matrixData)
println("height: ${matrix.rows}")
println("height: ${matrix.cols}")
println("height: ${matrix.isSquare}")
println("matrix[0, 0]: ${matrix[0, 0]}")
println("transposed matrix is:")
println("[${matrix.transpose()[0, 0]}, ${matrix.transpose()[0, 1]}, ${matrix.transpose()[0, 2]}]")
println("[${matrix.transpose()[1, 0]}, ${matrix.transpose()[1, 1]}, ${matrix.transpose()[1, 2]}]")
println("[${matrix.transpose()[2, 0]}, ${matrix.transpose()[2, 1]}, ${matrix.transpose()[2, 2]}]")

height: 3
height: 3
height: true
matrix[0, 0]: 1.0
transposed matrix is:
[1.0, 4.0, 7.0]
[2.0, 5.0, 8.0]
[3.0, 6.0, 9.0]


## 2.3.6 Smart Cast

In [2]:
interface Expr
class Num(val value: Int) : Expr
class Sum(val left: Expr, val right: Expr) : Expr

fun eval(e: Expr) : Int {
    if (e is Num) {
        val n = e as Num // 明示的なキャスト変換
        return n.value
    }
    if (e is Sum) {
        return eval(e.right) + eval(e.left) // ここでは変数はsmart castされている
    }
    throw IllegalArgumentException("Unknown expression")
}

fun main() {
    println(eval(Sum(Sum(Num(1), Num(2)), Num(4))))
    // 7
}

main()

7


Rafactoringする

In [3]:
interface Expr
class Num(val value: Int) : Expr
class Sum(val left: Expr, val right: Expr) : Expr

fun eval(e: Expr) : Int =
    if (e is Num) {
        e.value
    } else if (e is Sum) {
        eval(e.right) + eval(e.left)
    } else {
        throw IllegalArgumentException("Unknown expression")
    }

fun main() {
    println(eval(Sum(Sum(Num(1), Num(2)), Num(4))))
    // 7

    println(eval(Sum(Num(1), Num(2))))
    // 3
}

main()

7
3


さらにconsiceにできる

In [2]:
interface Expr
class Num(val value: Int) : Expr
class Sum(val left: Expr, val right: Expr) : Expr

fun eval(e: Expr) : Int =
   if (e is Num) e.value
   else if (e is Sum) eval(e.right) + eval(e.left)
   else throw IllegalArgumentException("Unknown expression")

fun main() {
    println(eval(Sum(Sum(Num(1), Num(2)), Num(4))))
    // 7

    println(eval(Sum(Num(1), Num(2))))
    // 3
}

main()

7
3


複数選択肢がある場合、もっといい書き方がある。つまり`when`

In [1]:
interface Expr
class Num(val value: Int) : Expr
class Sum(val left: Expr, val right: Expr) : Expr

fun eval(e: Expr) : Int =
    when (e) {
        is Num -> e.value // このブランチでは引数の型のスマートキャストをしている
        is Sum -> eval(e.right) + eval(e.left) // このブランチでは引数の型のスマートキャストをしている
        else -> throw IllegalArgumentException("Unknown expre")
    }

fun main() {
    println(eval(Sum(Sum(Num(1), Num(2)), Num(4))))
    // 7

    println(eval(Sum(Num(1), Num(2))))
    // 3
}

main()

7
3


Blocks as branches of if and when

In [3]:
interface Expr
class Num(val value: Int) : Expr
class Sum(val left: Expr, val right: Expr) : Expr

fun evalWithLogging(e: Expr): Int =
    when (e) {
        is Num -> {
            println("num: ${e.value}")
            e.value
        }
        is Sum -> {
            val left = evalWithLogging(e.left)
            val right = evalWithLogging(e.right)
            println("sum: $left + $right")
            left + right
        }
        else -> throw IllegalArgumentException("Unknown expression")
    }

fun main() {
    println(evalWithLogging(Sum(Sum(Num(1), Num(2)), Num(4))))
    //    num: 1
    //    num: 2
    //    sum: 1 + 2
    //    num: 4
    //    sum: 3 + 4
    //    7
}

main()

num: 1
num: 2
sum: 1 + 2
num: 4
sum: 3 + 4
7


## Iterating over things: while and for loops

Progression（プログレッション）とは、指定された範囲（Range）内で、一定のステップ（増分/減分）で値が連続的に変化する数列

In [5]:
fun fizzBuzz(i: Int) = when {
    i % 15 == 0 -> "FizzBuzz "
    i % 3 == 0 -> "Fizz "
    i % 5 == 0 -> "Buzz "
    else -> "$i "
 }

fun main() {
    for (i in 1..100) {
        println(fizzBuzz(i))
    }
}

main()

1 
2 
Fizz 
4 
Buzz 
Fizz 
7 
8 
Fizz 
Buzz 
11 
Fizz 
13 
14 
FizzBuzz 
16 
17 
Fizz 
19 
Buzz 
Fizz 
22 
23 
Fizz 
Buzz 
26 
Fizz 
28 
29 
FizzBuzz 
31 
32 
Fizz 
34 
Buzz 
Fizz 
37 
38 
Fizz 
Buzz 
41 
Fizz 
43 
44 
FizzBuzz 
46 
47 
Fizz 
49 
Buzz 
Fizz 
52 
53 
Fizz 
Buzz 
56 
Fizz 
58 
59 
FizzBuzz 
61 
62 
Fizz 
64 
Buzz 
Fizz 
67 
68 
Fizz 
Buzz 
71 
Fizz 
73 
74 
FizzBuzz 
76 
77 
Fizz 
79 
Buzz 
Fizz 
82 
83 
Fizz 
Buzz 
86 
Fizz 
88 
89 
FizzBuzz 
91 
92 
Fizz 
94 
Buzz 
Fizz 
97 
98 
Fizz 
Buzz 


In [8]:
fun fizzBuzz(i: Int) = when {
    i % 15 == 0 -> "FizzBuzz "
    i % 3 == 0 -> "Fizz "
    i % 5 == 0 -> "Buzz "
    else -> "$i "
}

fun main() {
    for (i in 100 downTo 1 step 2) {
        print(fizzBuzz(i))
    }
}

main()

Buzz 98 Fizz 94 92 FizzBuzz 88 86 Fizz 82 Buzz Fizz 76 74 Fizz Buzz 68 Fizz 64 62 FizzBuzz 58 56 Fizz 52 Buzz Fizz 46 44 Fizz Buzz 38 Fizz 34 32 FizzBuzz 28 26 Fizz 22 Buzz Fizz 16 14 Fizz Buzz 8 Fizz 4 2 

In [10]:
fun main() {
    val collection = listOf("red", "green", "blue")
    for (color in collection) {
        print("$color ")
    }
}

main()

red green blue 

In [13]:
fun main() {
    val binaryReps = mutableMapOf<Char, String>()
    for (char in 'A'..'F') {
        val binary = char.code.toString(radix=2) // ASCIIコードに変換する
        binaryReps[char] = binary
    }

    for ((letter, binary) in binaryReps) { // mapをイテレートできる
        println("$letter = $binary")
    }
}

main()

A = 1000001
B = 1000010
C = 1000011
D = 1000100
E = 1000101
F = 1000110


同じようなunpacking syntaxは現在のアイテムのインデックスにも適用できる
pythonのenumerateのようなもの

In [15]:
fun main() {
    val list = listOf("10", "11", "1001")
    for ((index, element) in list.withIndex()) {
        println("$index: $element")
    }
}

main()

0: 10
1: 11
2: 1001


`in`演算子でvalueがrangeの中にあるかどうか
`!in`演算子は上記の否定
例：userのvalidation, 文字やdigitsの数字


```kotlin
c in 'a'..'z'
```
は内部的には
```kotlin
'a' <= c && c <= 'z'
```

In [2]:
fun isLetter(c: Char) = c in 'a'..'z' || c in 'A'..'Z'
fun isNotDigit(c: Char) = c !in '0'..'9'

fun main() {
    println(isLetter('q'))
    println(isNotDigit('x'))
}

main()

true
true


`when` expressionでより便利な文字と数字のチェック

In [6]:
fun recognize(c: Char) = when(c) {
    in '0'..'9' -> "It's a digit!"
    in 'a'..'z', in 'A'..'Z' -> "It's a letter!"
    else -> "I don't know..."
}

fun main() {
    println(recognize('8'))
}

main()

It's a digit!


In [9]:
// compared alphabetically
fun main() {
    println("Kotlin" in "Java".."Scala")
    // true

    println("Java" in "Kotlin".."Scala")
    // false
}

main()

true
false


In [11]:
fun main() {
    println("Kotlin" in setOf("Java", "Scala"))
    // false
}

main()

false


## Throwing and catching exception in Kotlin

`throw` expressionなので、他のexpressionの一部として利用できる
`throw`が呼ばれると、この例では変数`percentage`が初期化されない。→ 8.1.7で詳細が語られる

In [3]:
val number = 99

val percentage =
    if (number in 0..100)
        number
    else
        throw IllegalArgumentException(
            "A percentage value must be between 0 and 100: $number"
        )

In [7]:
import java.io.BufferedReader
import java.io.StringReader
import java.text.NumberFormat

fun readNumber(reader: BufferedReader) : Int? {
    try {
        val line = reader.readLine()
        return Integer.parseInt(line)
    } catch (e: NumberFormatException) {
        return null
    } finally {
        reader.close()
    }
}

fun main() {
    val reader = BufferedReader(StringReader("239"))
    println(readNumber(reader))
    // 239

    val reader2 = BufferedReader(StringReader("asdf9"))
    println(readNumber(reader2))
    // null

    val reader3 = BufferedReader(StringReader("0000"))
    println(readNumber(reader3))
    // 0

    val reader4 = BufferedReader(StringReader("024"))
    println(readNumber(reader4))
    // 24
}

main()

239
null
0
24


In [3]:
import java.io.BufferedReader
import java.io.StringReader

fun readNumber(reader: BufferedReader) : Int {
    val line = reader.readLine()
    reader.close()
    return Integer.parseInt(line)
}

fun main() {
    val reader = BufferedReader(StringReader("239"))
    println(readNumber(reader))
    // 239

    // val reader1 = BufferedReader(StringReader("a"))
    // println(readNumber(reader1))
    // java.lang.NumberFormatException: For input string: "a"
}

main()

239


`try` expression
`if` exprとは違い、必ず中括弧で閉じないといけない

In [7]:
import java.io.BufferedReader
import java.io.StringReader

fun readNumber(reader: BufferedReader)  {
    val number = try {
        Integer.parseInt(reader.readLine())
    } catch (e: NumberFormatException) {
        null
    }
    println(number)
}

fun main() {
    val reader = BufferedReader(StringReader("not a number"))
    readNumber(reader)
    // null
}

main()

null
